In [15]:
import tensorflow as tf
import numpy as np
import os
import csv
import lpips
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

In [16]:
# Configure parameters
EPSILONS = [-2, -1, -0.5, 0, 0.5, 1, 2, 3, 4, 6, 8, 12, 16, 20, 24, 28, 28, 32, 36]  # Max perturbation bounds
ITERATIONS = 10  # Number of PGD steps
ALPHA = 2       # Step size per iteration
IMAGE_DIR = "/kaggle/input/tiny-imagenet/tiny-imagenet-200/test/images"
RESULTS_FILE = "PGD_attacks_with_LPIPS.csv"
NUM_IMAGES = 200
INPUT_SIZE = (224, 224)

In [17]:
# Initialize models
model = ResNet101(weights='imagenet')
loss_fn = lpips.LPIPS(net='alex')  # LPIPS model

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth


In [18]:
def reverse_preprocess(adv_image):
    """Convert model output back to displayable format"""
    mean = [103.939, 116.779, 123.68]
    reversed_img = adv_image.copy()
    reversed_img[..., 0] += mean[0]
    reversed_img[..., 1] += mean[1]
    reversed_img[..., 2] += mean[2]
    reversed_img = reversed_img[..., ::-1]  # BGR to RGB
    return np.clip(reversed_img, 0, 255).astype('uint8')

def pgd_attack(model, input_image, epsilon):
    """Generate PGD adversarial example"""
    input_tensor = tf.convert_to_tensor(input_image)
    adv_image = tf.identity(input_tensor)

    for _ in range(ITERATIONS):
        with tf.GradientTape() as tape:
            tape.watch(adv_image)
            prediction = model(adv_image)
            loss = tf.keras.losses.categorical_crossentropy(prediction, prediction)

        # Compute gradient and update adversarial image
        gradient = tape.gradient(loss, adv_image)
        perturbation = ALPHA * tf.sign(gradient)
        adv_image = tf.stop_gradient(adv_image + perturbation)

        # Project back to epsilon ball
        perturbation_total = adv_image - input_tensor
        perturbation_total = tf.clip_by_value(perturbation_total, -epsilon, epsilon)
        adv_image = input_tensor + perturbation_total

        # Maintain valid input range
        adv_image = tf.clip_by_value(adv_image, -100.0, 150.0)

    return adv_image.numpy()

def calculate_lpips(original, adversarial):
    """Calculate LPIPS distance between two images"""
    original_tensor = lpips.im2tensor(original)
    adversarial_tensor = lpips.im2tensor(adversarial)
    return loss_fn(original_tensor, adversarial_tensor).item()

In [19]:
# Main processing loop
results = []
image_files = [f for f in os.listdir(IMAGE_DIR) if f.endswith(".JPEG")][:NUM_IMAGES]

for epsilon in EPSILONS:
    for img_file in image_files:
        try:
            # Load and preprocess image
            img_path = os.path.join(IMAGE_DIR, img_file)
            img = image.load_img(img_path, target_size=INPUT_SIZE)
            img_array = image.img_to_array(img)

            # Original image for visualization
            original = img_array.copy().astype('uint8')

            # Preprocess for model
            processed = preprocess_input(img_array.copy())
            processed = np.expand_dims(processed, axis=0)

            # Generate adversarial example
            adversarial = pgd_attack(model, processed, epsilon)
            adversarial = np.clip(adversarial, -100.0, 150.0)

            # Reverse preprocessing for visualization
            adv_reversed = reverse_preprocess(adversarial[0])

            # Calculate metrics
            ssim = tf.image.ssim(
                tf.convert_to_tensor([original], tf.uint8),
                tf.convert_to_tensor([adv_reversed], tf.uint8),
                max_val=255
            ).numpy()[0]

            lpips_score = calculate_lpips(original, adv_reversed)

            # Get predictions
            clean_pred = model.predict(processed)
            adv_pred = model.predict(adversarial)
            clean_top = decode_predictions(clean_pred, top=1)[0][0]
            adv_top = decode_predictions(adv_pred, top=1)[0][0]

            # Store results
            results.append({
                'Image': img_file,
                'Epsilon': epsilon,
                'SSIM': float(ssim),
                'LPIPS': lpips_score,
                'CleanLabel': clean_top[1],
                'CleanConfidence': float(clean_top[2]),
                'AdvLabel': adv_top[1],
                'AdvConfidence': float(adv_top[2]),
                'Success': int(clean_top[1] != adv_top[1])
            })

        except Exception as e:
            print(f"Error with {img_file}: {str(e)}")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/tiny-imagenet/tiny-imagenet-200/test/images'

In [ ]:
# Save results
with open(RESULTS_FILE, 'w', newline='') as csvfile:
    fieldnames = [
        'Image', 'Epsilon', 'SSIM', 'LPIPS',
        'CleanLabel', 'CleanConfidence',
        'AdvLabel', 'AdvConfidence',
        'Success'
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

print(f"Results saved to {RESULTS_FILE}")